# DocuShield QuickSight ETL - FINAL FIXED VERSION V2
## Generate Exact Dataset Structure for QuickSight Analytics

This notebook creates the three specific datasets with completely fixed SQL handling:
1. **DocumentAgg** - One row per contract with aggregated metrics
2. **RiskFindings** - Many rows per contract with individual findings
3. **UserActivityDaily** - One row per user per day with activity metrics

**FINAL FIXES APPLIED:**
- Uses existing schema (confidence column only)
- NO % characters in SQL to avoid format errors
- Direct SQLAlchemy execution
- Completely error-proof SQL queries
- Timestamps converted to strings for Athena compatibility
- NaN values filled to prevent Parquet issues
- **FIXED: Proper Hive partitioning (year=2024/month=10/) for Athena compatibility**

**🤖 AUTO-TRIGGER INTEGRATION:**
- Checks S3 for trigger files from backend: `s3://bucket/docushield/triggers/etl_trigger_*`
- Automatically runs when new documents are processed
- Cleans up trigger files after successful execution
- Falls back to manual mode if no triggers found
- Backend integration via `auto_export_service.py`

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os
import logging
from datetime import datetime, timedelta
import json
import boto3
from io import StringIO, BytesIO
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🚀 DocuShield QuickSight ETL - FINAL FIXED VERSION V2 (AUTO-TRIGGER ENABLED)")
print("📅 Started at:", datetime.now())
print("🔧 Fixed: Proper Hive partitioning for Athena compatibility")
print("🤖 Auto-trigger functionality: ENABLED")

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


🚀 DocuShield QuickSight ETL - FINAL FIXED VERSION V2 (AUTO-TRIGGER ENABLED)
📅 Started at: 2025-10-19 18:37:34.418615
🔧 Fixed: Proper Hive partitioning for Athena compatibility
🤖 Auto-trigger functionality: ENABLED


## 🤖 Auto-Trigger Detection System

This cell checks for trigger files from the backend to determine if ETL should run automatically.

In [2]:
# Auto-Trigger Detection System
def check_for_triggers():
    """Check S3 for new trigger files from backend"""
    try:
        s3 = boto3.client('s3')
        
        # S3 configuration for triggers
        trigger_bucket = 'sagemaker-us-east-1-192933326034'
        trigger_prefix = 'docushield/triggers/etl_trigger_'
        
        print(f"🔍 Checking for ETL triggers...")
        print(f"📦 S3 Trigger Bucket: {trigger_bucket}")
        print(f"📁 Trigger Prefix: {trigger_prefix}")
        
        # List files in triggers folder
        response = s3.list_objects_v2(
            Bucket=trigger_bucket,
            Prefix=trigger_prefix
        )
        
        if 'Contents' in response and len(response['Contents']) > 0:
            # Found trigger files! Get the most recent one
            trigger_files = sorted(response['Contents'], key=lambda x: x['LastModified'], reverse=True)
            trigger_file = trigger_files[0]['Key']
            
            print(f"🚀 ETL trigger found: {trigger_file}")
            
            # Download and read trigger info
            obj = s3.get_object(Bucket=trigger_bucket, Key=trigger_file)
            trigger_info = json.loads(obj['Body'].read())
            
            print(f"📋 Trigger Details:")
            print(f"   🆔 Execution ID: {trigger_info.get('execution_id', 'N/A')}")
            print(f"   📄 Contract ID: {trigger_info.get('contract_id', 'N/A')}")
            print(f"   👤 User ID: {trigger_info.get('user_id', 'N/A')}")
            print(f"   ⏰ Timestamp: {trigger_info.get('timestamp', 'N/A')}")
            print(f"   🔧 Trigger Type: {trigger_info.get('trigger_type', 'document_processed')}")
            
            return trigger_info, trigger_file
        else:
            print("ℹ️ No ETL triggers found - running in manual mode")
            return None, None
            
    except Exception as e:
        print(f"⚠️ Error checking for triggers: {e}")
        print("📝 Continuing in manual mode...")
        return None, None

def cleanup_trigger_file(trigger_file):
    """Clean up processed trigger file"""
    if trigger_file:
        try:
            s3 = boto3.client('s3')
            trigger_bucket = 'sagemaker-us-east-1-192933326034'
            
            s3.delete_object(Bucket=trigger_bucket, Key=trigger_file)
            print(f"🗑️ Cleaned up trigger file: {trigger_file}")
        except Exception as e:
            print(f"⚠️ Failed to cleanup trigger file: {e}")

# Check for triggers at startup
trigger_info, trigger_file = check_for_triggers()

# Set execution mode
if trigger_info:
    execution_mode = "TRIGGERED"
    print(f"\n🤖 EXECUTION MODE: {execution_mode}")
    print(f"🚀 Running ETL because new document was processed!")
else:
    execution_mode = "MANUAL"
    print(f"\n📋 EXECUTION MODE: {execution_mode}")
    print(f"🔧 Running ETL in manual mode")

print("✅ Trigger system initialized successfully")

🔍 Checking for ETL triggers...
📦 S3 Trigger Bucket: sagemaker-us-east-1-192933326034
📁 Trigger Prefix: docushield/triggers/etl_trigger_
✅ Trigger system initialized successfully


In [ ]:
# TiDB Connection Configuration
TIDB_HOST = ""
TIDB_PORT = ""
TIDB_USER = ""
TIDB_PASSWORD = ""
TIDB_DATABASE = ""

# S3 Output Configuration
OUTPUT_S3_BUCKET = ""
OUTPUT_S3_PREFIX = ""
OUTPUT_S3 = f"s3://{OUTPUT_S3_BUCKET}/{OUTPUT_S3_PREFIX}"

# Build TiDB connection string
DATABASE_URL = f"mysql+pymysql://{TIDB_USER}:{TIDB_PASSWORD}@{TIDB_HOST}:{TIDB_PORT}/{TIDB_DATABASE}?ssl_verify_cert=false&ssl_verify_identity=false"

print(f"🔗 Connecting to TiDB: {TIDB_HOST}:{TIDB_PORT}/{TIDB_DATABASE}")
print(f"📦 S3 Output: {OUTPUT_S3}")

engine = create_engine(DATABASE_URL)

def execute_sql(sql_query):
    """Execute SQL query using SQLAlchemy text() to avoid format issues"""
    try:
        with engine.connect() as connection:
            result = connection.execute(text(sql_query))
            # Fetch ALL results - no limits
            rows = result.fetchall()
            columns = result.keys()
            # Convert to DataFrame
            df = pd.DataFrame(rows, columns=columns)
            print(f"📊 SQL executed: Retrieved {len(df)} rows")
            return df
    except Exception as e:
        logger.error(f"SQL execution failed: {e}")
        logger.error(f"Query: {sql_query}")
        raise

# Test connection
try:
    test_result = execute_sql("SELECT 1 as test")
    print("✅ TiDB connection established successfully")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

🔗 Connecting to TiDB: gateway01.us-east-1.prod.aws.tidbcloud.com:4000/docushield_dev
📦 S3 Output: s3://sagemaker-us-east-1-192933326034/docushield/analytics
📊 SQL executed: Retrieved 1 rows
✅ TiDB connection established successfully


## 📊 Dataset 1: DocumentAgg (One row per contract)

In [3]:
print("📊 Creating DocumentAgg dataset...")

# Build DocumentAgg with existing schema - NO PERCENT SIGNS
document_agg_sql = """
SELECT 
    bc.contract_id,
    bc.created_at,
    bc.updated_at,
    bc.document_type,
    bc.industry_type,
    bc.document_category,
    bc.owner_user_id,
    ROUND(COALESCE(bc.file_size, 0) / (1024.0 * 1024.0), 3) as file_size_mb,
    
    COALESCE(latest_run.processing_time_seconds, 0) as processing_time_seconds,
    COALESCE(risk_metrics.doc_max_risk, 0) as doc_max_risk,
    COALESCE(risk_metrics.risk_count, 0) as risk_count,
    
    CASE WHEN liability_check.has_liability > 0 THEN 1 ELSE 0 END as has_unlimited_liability_doc,
    CASE WHEN renewal_check.has_renewal > 0 THEN 1 ELSE 0 END as has_auto_renewal_doc,
    COALESCE(high_conf.avg_conf_high_doc, 0) as avg_conf_high_doc
    
FROM bronze_contracts bc

LEFT JOIN (
    SELECT 
        contract_id,
        TIMESTAMPDIFF(SECOND, started_at, completed_at) as processing_time_seconds,
        ROW_NUMBER() OVER (PARTITION BY contract_id ORDER BY completed_at DESC) as rn
    FROM processing_runs 
    WHERE status = 'completed' 
      AND started_at IS NOT NULL 
      AND completed_at IS NOT NULL
) latest_run ON bc.contract_id = latest_run.contract_id AND latest_run.rn = 1

LEFT JOIN (
    SELECT 
        contract_id,
        MAX(confidence * 100.0) as doc_max_risk,
        COUNT(finding_id) as risk_count
    FROM gold_findings
    GROUP BY contract_id
) risk_metrics ON bc.contract_id = risk_metrics.contract_id

LEFT JOIN (
    SELECT 
        contract_id,
        COUNT(*) as has_liability
    FROM gold_findings
    WHERE finding_type LIKE 'liability' OR finding_type LIKE 'unlimited'
    GROUP BY contract_id
) liability_check ON bc.contract_id = liability_check.contract_id

LEFT JOIN (
    SELECT 
        contract_id,
        COUNT(*) as has_renewal
    FROM gold_findings
    WHERE finding_type LIKE 'renewal' OR finding_type LIKE 'auto'
    GROUP BY contract_id
) renewal_check ON bc.contract_id = renewal_check.contract_id

LEFT JOIN (
    SELECT 
        contract_id,
        AVG(confidence) as avg_conf_high_doc
    FROM gold_findings
    WHERE LOWER(severity) = 'high'
    GROUP BY contract_id
) high_conf ON bc.contract_id = high_conf.contract_id

ORDER BY bc.created_at DESC
"""

document_agg = execute_sql(document_agg_sql)

# Fix timestamps for Athena compatibility
if len(document_agg) > 0:
    # Convert timestamp columns to strings
    if 'created_at' in document_agg.columns:
        document_agg['created_at'] = pd.to_datetime(document_agg['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
    if 'updated_at' in document_agg.columns:
        document_agg['updated_at'] = pd.to_datetime(document_agg['updated_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
    
    # Fill NaN values
    document_agg = document_agg.fillna({
        'document_type': '',
        'industry_type': '',
        'document_category': '',
        'owner_user_id': '',
        'file_size_mb': 0.0,
        'processing_time_seconds': 0,
        'doc_max_risk': 0.0,
        'risk_count': 0,
        'has_unlimited_liability_doc': 0,
        'has_auto_renewal_doc': 0,
        'avg_conf_high_doc': 0.0
    })

     # Fix floating point precision issues for parquet compatibility
    document_agg['doc_max_risk'] = document_agg['doc_max_risk'].round(2).astype('float64')
    document_agg['avg_conf_high_doc'] = document_agg['avg_conf_high_doc'].round(4).astype('float64')
    document_agg['file_size_mb'] = document_agg['file_size_mb'].round(3).astype('float64')
    
    # Ensure integer columns are proper integers
    document_agg['processing_time_seconds'] = document_agg['processing_time_seconds'].astype('int64')
    document_agg['risk_count'] = document_agg['risk_count'].astype('int64')
    document_agg['has_unlimited_liability_doc'] = document_agg['has_unlimited_liability_doc'].astype('int32')
    document_agg['has_auto_renewal_doc'] = document_agg['has_auto_renewal_doc'].astype('int32')

print(f"✅ DocumentAgg created: {len(document_agg)} records")
print(f"📊 Columns: {list(document_agg.columns)}")
if len(document_agg) > 0:
    print(f"📈 Max risk score: {document_agg['doc_max_risk'].max():.1f}")
    print(f"📊 Avg file size (MB): {document_agg['file_size_mb'].mean():.2f}")
    print(f"🔧 Timestamps converted to strings for Athena compatibility")
document_agg.head()

📊 Creating DocumentAgg dataset...
📊 SQL executed: Retrieved 3 rows
✅ DocumentAgg created: 3 records
📊 Columns: ['contract_id', 'created_at', 'updated_at', 'document_type', 'industry_type', 'document_category', 'owner_user_id', 'file_size_mb', 'processing_time_seconds', 'doc_max_risk', 'risk_count', 'has_unlimited_liability_doc', 'has_auto_renewal_doc', 'avg_conf_high_doc']
📈 Max risk score: 90.0
📊 Avg file size (MB): 0.17
🔧 Timestamps converted to strings for Athena compatibility


,contract_id,created_at,updated_at,document_type,industry_type,document_category,owner_user_id,file_size_mb,processing_time_seconds,doc_max_risk,risk_count,has_unlimited_liability_doc,has_auto_renewal_doc,avg_conf_high_doc
0,01b11ef1-13c4-45bf-89e8-72958197f161,2025-10-10 23:22:42,2025-10-10 23:22:42,,,contract,7790d9af-65ea-48ea-852a-7c847a522d25,0.237,71,90.0,17,0,0,0.8571
1,16630e5d-948b-4c56-999c-910191be7c9f,2025-10-08 19:58:48,2025-10-08 19:58:48,,,contract,7790d9af-65ea-48ea-852a-7c847a522d25,0.155,72,0.0,0,0,0,0.0000
2,788673ff-0487-4f7f-99d2-4bd637bdf382,2025-10-08 14:37:14,2025-10-08 14:37:14,,,contract,7790d9af-65ea-48ea-852a-7c847a522d25,0.118,98,0.0,0,0,0,0.0000


## 🔍 Dataset 2: RiskFindings (Many rows per contract)

In [4]:
print("🔍 Creating RiskFindings dataset...")

# Build RiskFindings with existing schema - NO PERCENT SIGNS
risk_findings_sql = """
SELECT 
    finding_id,
    contract_id,
    span_id,
    finding_type,
    LOWER(severity) as severity_level,
    confidence * 100.0 as risk_score,
    confidence as confidence_score,
    title,
    description,
    impact_category,
    estimated_impact,
    detection_method,
    model_version,
    created_at,
    DATE(created_at) as created_date
FROM gold_findings
WHERE finding_id IS NOT NULL
"""

risk_findings = execute_sql(risk_findings_sql)

# Fix timestamps and add partitioning columns for Athena compatibility
if len(risk_findings) > 0:
    # Convert timestamp columns to strings
    if 'created_at' in risk_findings.columns:
        risk_findings['created_at'] = pd.to_datetime(risk_findings['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
    if 'created_date' in risk_findings.columns:
        risk_findings['created_date'] = pd.to_datetime(risk_findings['created_date']).dt.strftime('%Y-%m-%d')
        # Add partitioning columns
        risk_findings['year'] = pd.to_datetime(risk_findings['created_date']).dt.year
        risk_findings['month'] = pd.to_datetime(risk_findings['created_date']).dt.month
    
    # Fill NaN values
    risk_findings = risk_findings.fillna({
        'finding_type': '',
        'severity_level': '',
        'title': '',
        'description': '',
        'impact_category': '',
        'estimated_impact': '',
        'detection_method': '',
        'model_version': '',
        'risk_score': 0.0,
        'confidence_score': 0.0
    })

print(f"✅ RiskFindings created: {len(risk_findings)} records")
print(f"📊 Columns: {list(risk_findings.columns)}")
if len(risk_findings) > 0:
    print(f"📅 Date range: {risk_findings['created_date'].min()} to {risk_findings['created_date'].max()}")
    print(f"🎯 Severity distribution: {dict(risk_findings['severity_level'].value_counts())}")
    print(f"📊 Risk score range: {risk_findings['risk_score'].min():.1f} - {risk_findings['risk_score'].max():.1f}")
    print(f"🔧 Timestamps converted to strings for Athena compatibility")
risk_findings.head()

🔍 Creating RiskFindings dataset...
📊 SQL executed: Retrieved 17 rows
✅ RiskFindings created: 17 records
📊 Columns: ['finding_id', 'contract_id', 'span_id', 'finding_type', 'severity_level', 'risk_score', 'confidence_score', 'title', 'description', 'impact_category', 'estimated_impact', 'detection_method', 'model_version', 'created_at', 'created_date', 'year', 'month']
📅 Date range: 2025-10-10 to 2025-10-10
🎯 Severity distribution: {'medium': 9, 'high': 7, 'info': 1}
📊 Risk score range: 70.0 - 90.0
🔧 Timestamps converted to strings for Athena compatibility


,finding_id,contract_id,span_id,finding_type,severity_level,risk_score,confidence_score,title,description,impact_category,estimated_impact,detection_method,model_version,created_at,created_date,year,month
0,1ebf9894-d7df-41ca-ab6d-6ec793b95c64,01b11ef1-13c4-45bf-89e8-72958197f161,None,risk_analysis,high,89.999998,0.9,Immediate action clauses (2 instances),Found 2 instances of immediate action clauses,,,orchestrator,2.0.0,2025-10-10 23:23:29,2025-10-10,2025,10
1,322fe68f-74f1-4ae9-b1c6-24f9fe68e6e5,01b11ef1-13c4-45bf-89e8-72958197f161,None,query_insight,info,89.999998,0.9,Query Analysis: Perform comprehensive document...,Found 4 high-priority items related to your query,,,orchestrator,2.0.0,2025-10-10 23:23:29,2025-10-10,2025,10
2,40d6475e-3c45-4973-87ea-45f6aa2500f0,01b11ef1-13c4-45bf-89e8-72958197f161,None,overall_risk_assessment,medium,89.999998,0.9,Overall Risk Assessment: Medium,Risk analysis identified 4 risk factors,,,orchestrator,2.0.0,2025-10-10 23:23:29,2025-10-10,2025,10
3,40f0f8af-12ac-42ea-9958-9479a690df0c,01b11ef1-13c4-45bf-89e8-72958197f161,None,liability_unlimited,high,80.000001,0.8,Unlimited liability clause detected,"Found matches: ['without limit', 'without limi...",,,ai,1.0.0,2025-10-10 23:23:52,2025-10-10,2025,10
4,59ec2681-a1d7-4c7f-a7ee-eddeadd540a3,01b11ef1-13c4-45bf-89e8-72958197f161,None,risk_analysis,medium,89.999998,0.9,Exclusive rights or obligations (22 instances),Found 22 instances of exclusive rights or obli...,,,orchestrator,2.0.0,2025-10-10 23:23:29,2025-10-10,2025,10


## 👥 Dataset 3: UserActivityDaily (One row per user per day)

In [5]:
print("👥 Creating UserActivityDaily dataset...")

# Try complex query first, fallback to simple if timeout
user_activity_sql = """
SELECT 
    DATE(bc.created_at) as activity_date,
    bc.owner_user_id as user_id,
    COUNT(DISTINCT bc.contract_id) as documents_processed,
    COALESCE(AVG(daily_risk.doc_max_risk), 0) as avg_risk_score,
    (COUNT(DISTINCT bc.contract_id) * 10 + (100 - COALESCE(AVG(daily_risk.doc_max_risk), 0))) as productivity_score,
    u.name as user_name,
    u.email
    
FROM bronze_contracts bc

LEFT JOIN users u ON bc.owner_user_id = u.user_id

LEFT JOIN (
    SELECT 
        contract_id,
        MAX(confidence * 100.0) as doc_max_risk
    FROM gold_findings
    GROUP BY contract_id
) daily_risk ON bc.contract_id = daily_risk.contract_id

WHERE bc.created_at IS NOT NULL
GROUP BY DATE(bc.created_at), bc.owner_user_id, u.name, u.email
ORDER BY activity_date DESC, documents_processed DESC
"""

try:
    user_activity_daily = execute_sql(user_activity_sql)
except Exception as e:
    print(f"⚠️ Complex query failed: {e}")
    print("🔄 Falling back to simplified query...")
    
    # Simplified fallback query
    simple_sql = """
    SELECT 
        DATE(bc.created_at) as activity_date,
        bc.owner_user_id as user_id,
        COUNT(DISTINCT bc.contract_id) as documents_processed,
        50.0 as avg_risk_score,
        (COUNT(DISTINCT bc.contract_id) * 10 + 50) as productivity_score,
        COALESCE(u.name, 'Unknown User') as user_name,
        COALESCE(u.email, 'unknown@example.com') as email
    FROM bronze_contracts bc
    LEFT JOIN users u ON bc.owner_user_id = u.user_id
    WHERE bc.created_at IS NOT NULL
    GROUP BY DATE(bc.created_at), bc.owner_user_id, u.name, u.email
    ORDER BY activity_date DESC
    LIMIT 1000
    """
    user_activity_daily = execute_sql(simple_sql)

# Fix timestamps and add partitioning columns for Athena compatibility
if len(user_activity_daily) > 0:
    # Convert activity_date to datetime first, then to string for Athena
    user_activity_daily['activity_date'] = pd.to_datetime(user_activity_daily['activity_date'])
    user_activity_daily['year'] = user_activity_daily['activity_date'].dt.year
    user_activity_daily['month'] = user_activity_daily['activity_date'].dt.month
    # Convert to string format for Athena
    user_activity_daily['activity_date'] = user_activity_daily['activity_date'].dt.strftime('%Y-%m-%d')
    
    # Fill NaN values
    user_activity_daily = user_activity_daily.fillna({
        'user_name': 'Unknown User',
        'email': 'unknown@example.com',
        'documents_processed': 0,
        'avg_risk_score': 0.0,
        'productivity_score': 0.0
    })

print(f"✅ UserActivityDaily created: {len(user_activity_daily)} records")
print(f"📊 Columns: {list(user_activity_daily.columns)}")
if len(user_activity_daily) > 0:
    print(f"📅 Date range: {user_activity_daily['activity_date'].min()} to {user_activity_daily['activity_date'].max()}")
    print(f"👥 Unique users: {user_activity_daily['user_id'].nunique()}")
    print(f"📊 Avg documents per day: {user_activity_daily['documents_processed'].mean():.1f}")
    print(f"🏆 Avg productivity score: {user_activity_daily['productivity_score'].mean():.1f}")
    print(f"🔧 Timestamps converted to strings for Athena compatibility")
user_activity_daily.head()

👥 Creating UserActivityDaily dataset...
📊 SQL executed: Retrieved 2 rows
✅ UserActivityDaily created: 2 records
📊 Columns: ['activity_date', 'user_id', 'documents_processed', 'avg_risk_score', 'productivity_score', 'user_name', 'email', 'year', 'month']
📅 Date range: 2025-10-08 to 2025-10-10
👥 Unique users: 1
📊 Avg documents per day: 1.5
🏆 Avg productivity score: 70.0
🔧 Timestamps converted to strings for Athena compatibility


,activity_date,user_id,documents_processed,avg_risk_score,productivity_score,user_name,email,year,month
0,2025-10-10,7790d9af-65ea-48ea-852a-7c847a522d25,1,89.999998,20.000002,chakri k,chakradhark1@outlook.com,2025,10
1,2025-10-08,7790d9af-65ea-48ea-852a-7c847a522d25,2,0.000000,120.000000,chakri k,chakradhark1@outlook.com,2025,10


## 📦 Export to S3 (Parquet + CSV) - FIXED PARTITIONING

In [6]:
# Initialize S3 client
s3_client = boto3.client('s3')
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

def upload_df_to_s3_parquet_fixed(df, dataset_name, description, partition_cols=None):
    """Upload DataFrame to S3 with proper Hive partitioning for Athena compatibility"""
    if len(df) == 0:
        print(f"⚠️ Skipping {description} - no data")
        return None
    
    try:
        # Fix DataFrame for Athena compatibility
        df_fixed = df.copy()
        
        # Convert timestamp columns to strings for Athena compatibility
        timestamp_cols = ['created_at', 'updated_at', 'activity_date', 'created_date']
        for col in timestamp_cols:
            if col in df_fixed.columns:
                df_fixed[col] = pd.to_datetime(df_fixed[col]).dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Fill NaN values to avoid Parquet issues
        for col in df_fixed.columns:
            if df_fixed[col].dtype == 'object':
                df_fixed[col] = df_fixed[col].fillna('')
            elif df_fixed[col].dtype in ['float64', 'int64']:
                df_fixed[col] = df_fixed[col].fillna(0)
        
        # S3 path for data
        s3_path = f"s3://{OUTPUT_S3_BUCKET}/{OUTPUT_S3_PREFIX}/parquet/{dataset_name}/"
        
        # Write with proper Hive partitioning
        if partition_cols:
            print(f"🔧 Creating Hive partitions for {dataset_name}...")
            
            # Group by partition columns
            grouped = df_fixed.groupby(partition_cols)
            
            for partition_values, group_df in grouped:
                # Handle single partition column
                if len(partition_cols) == 1:
                    partition_values = [partition_values]
                
                # Create Hive-style partition path: year=2024/month=10/
                partition_path = "/".join([f"{col}={val}" for col, val in zip(partition_cols, partition_values)])
                
                # Remove partition columns from the data (Athena adds them automatically)
                data_df = group_df.drop(columns=partition_cols)
                
                # Convert to PyArrow Table
                table = pa.Table.from_pandas(data_df)
                
                # Write to S3 with proper Hive partition structure
                buffer = BytesIO()
                pq.write_table(table, buffer)
                buffer.seek(0)
                
                # Upload to S3 with Hive partition path
                s3_key = f"{OUTPUT_S3_PREFIX}/parquet/{dataset_name}/{partition_path}/data.parquet"
                s3_client.put_object(
                    Bucket=OUTPUT_S3_BUCKET,
                    Key=s3_key,
                    Body=buffer.getvalue(),
                    ContentType='application/octet-stream'
                )
                
                print(f"✅ Uploaded partition {partition_path}: {len(group_df)} records")
        
        else:
            # Single Parquet file for non-partitioned tables
            table = pa.Table.from_pandas(df_fixed)
            pq.write_table(table, f"{s3_path}data.parquet")
            print(f"✅ Uploaded single file: {len(df_fixed)} records")
        
        print(f"✅ {description}: {s3_path} ({len(df)} records)")
        return s3_path
        
    except Exception as e:
        print(f"❌ Failed to upload {description}: {e}")
        import traceback
        traceback.print_exc()
        return None

def upload_df_to_s3_csv(df, filename, description):
    """Upload DataFrame to S3 as CSV (legacy support)"""
    if len(df) == 0:
        print(f"⚠️ Skipping {description} - no data")
        return None
    
    try:
        # Convert DataFrame to CSV string
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)
        
        # Upload to S3
        s3_key = f"{OUTPUT_S3_PREFIX}/csv/{filename}"
        s3_client.put_object(
            Bucket=OUTPUT_S3_BUCKET,
            Key=s3_key,
            Body=csv_buffer.getvalue(),
            ContentType='text/csv'
        )
        
        s3_url = f"s3://{OUTPUT_S3_BUCKET}/{s3_key}"
        print(f"✅ {description}: {s3_url} ({len(df)} records)")
        return s3_url
        
    except Exception as e:
        print(f"❌ Failed to upload {description}: {e}")
        return None

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [7]:
# Export the three main datasets with FIXED partitioning
print("📦 EXPORTING TO PARQUET (Athena-optimized with FIXED Hive partitioning):")
parquet_files = {}

# 1. DocumentAgg - Primary dataset (no partitioning needed - small dataset)
parquet_files['document_agg'] = upload_df_to_s3_parquet_fixed(
    document_agg, 
    'document_agg',
    'DocumentAgg Dataset (Parquet for Athena)'
)

# 2. RiskFindings - Partitioned by year/month with PROPER Hive format
parquet_files['risk_findings'] = upload_df_to_s3_parquet_fixed(
    risk_findings, 
    'risk_findings',
    'RiskFindings Dataset (Parquet for Athena)',
    partition_cols=['year', 'month']
)

# 3. UserActivityDaily - Partitioned by year/month with PROPER Hive format
parquet_files['user_activity'] = upload_df_to_s3_parquet_fixed(
    user_activity_daily, 
    'user_activity_daily',
    'UserActivityDaily Dataset (Parquet for Athena)',
    partition_cols=['year', 'month']
)

print("\n📄 EXPORTING TO CSV (Legacy support):")

# Also export CSV versions for backward compatibility
exported_files = {}
exported_files['document_agg'] = upload_df_to_s3_csv(
    document_agg, 
    'DocuShield_Contracts.csv',
    'DocumentAgg Dataset (CSV)'
)

exported_files['risk_findings'] = upload_df_to_s3_csv(
    risk_findings, 
    'DocuShield_Risks.csv',
    'RiskFindings Dataset (CSV)'
)

exported_files['user_activity'] = upload_df_to_s3_csv(
    user_activity_daily, 
    'DocuShield_Activity.csv',
    'UserActivityDaily Dataset (CSV)'
)

📦 EXPORTING TO PARQUET (Athena-optimized with FIXED Hive partitioning):
✅ Uploaded single file: 3 records
✅ DocumentAgg Dataset (Parquet for Athena): s3://sagemaker-us-east-1-192933326034/docushield/analytics/parquet/document_agg/ (3 records)
🔧 Creating Hive partitions for risk_findings...
✅ Uploaded partition year=2025/month=10: 17 records
✅ RiskFindings Dataset (Parquet for Athena): s3://sagemaker-us-east-1-192933326034/docushield/analytics/parquet/risk_findings/ (17 records)
🔧 Creating Hive partitions for user_activity_daily...
✅ Uploaded partition year=2025/month=10: 2 records
✅ UserActivityDaily Dataset (Parquet for Athena): s3://sagemaker-us-east-1-192933326034/docushield/analytics/parquet/user_activity_daily/ (2 records)

📄 EXPORTING TO CSV (Legacy support):
✅ DocumentAgg Dataset (CSV): s3://sagemaker-us-east-1-192933326034/docushield/analytics/csv/DocuShield_Contracts.csv (3 records)
✅ RiskFindings Dataset (CSV): s3://sagemaker-us-east-1-192933326034/docushield/analytics/csv/Do

## 📊 Final Summary Report

In [8]:
print("\n" + "=" * 80)
print("🎉 FINAL FIXED QuickSight datasets exported successfully!")
print(f"📅 Export timestamp: {timestamp}")
print(f"🤖 Execution mode: {execution_mode}")
if trigger_info:
    print(f"🚀 Triggered by: Contract {trigger_info.get('contract_id', 'N/A')[:8]}... (User: {trigger_info.get('user_id', 'N/A')[:8]}...)")
print(f"🔧 FIXED: Proper Hive partitioning (year=2024/month=10/) for Athena")
print(f"🛡️ Format safety: ✅ No percent characters in SQL")
print(f"🔧 Athena compatibility: ✅ Timestamps converted to strings")

print("\n📋 EXPORT SUMMARY:")
datasets = [
    ('DocumentAgg', document_agg, 'One row per contract with aggregated metrics'),
    ('RiskFindings', risk_findings, 'Many rows per contract with individual findings'),
    ('UserActivityDaily', user_activity_daily, 'One row per user per day with activity metrics')
]

for name, df, desc in datasets:
    if len(df) > 0:
        print(f"   ✅ {name}: {len(df)} records - {desc}")
    else:
        print(f"   ⚠️ {name}: No data found")

print("\n🚀 ETL PIPELINE COMPLETED SUCCESSFULLY!")
print("🎯 Data is ready for Athena with proper Hive partitioning")
print("📊 Risk scores calculated from confidence column (0-100 scale)")
print("🔒 SQL format errors completely eliminated")
print("🔧 Partition naming issue FIXED - now uses year=2024/month=10/ format")

print("\n🔗 Next steps:")
print("   1. Run MSCK REPAIR TABLE commands in Athena")
print("   2. Test queries: SELECT COUNT(*) FROM docushield_analytics.risk_findings")
print("   3. Create QuickSight datasets using these tables")
print("   4. Build your analytics dashboards")

# Clean up trigger file if this was a triggered execution
if execution_mode == "TRIGGERED" and trigger_file:
    print(f"\n🧹 CLEANUP:")
    cleanup_trigger_file(trigger_file)
    print(f"✅ Trigger-based execution completed and cleaned up")
elif execution_mode == "MANUAL":
    print(f"\n📋 Manual execution completed - no cleanup needed")


🎉 FINAL FIXED QuickSight datasets exported successfully!
📅 Export timestamp: 20251019_183735
🔧 FIXED: Proper Hive partitioning (year=2024/month=10/) for Athena
🛡️ Format safety: ✅ No percent characters in SQL
🔧 Athena compatibility: ✅ Timestamps converted to strings

📋 EXPORT SUMMARY:
   ✅ DocumentAgg: 3 records - One row per contract with aggregated metrics
   ✅ RiskFindings: 17 records - Many rows per contract with individual findings
   ✅ UserActivityDaily: 2 records - One row per user per day with activity metrics

🚀 ETL PIPELINE COMPLETED SUCCESSFULLY!
🎯 Data is ready for Athena with proper Hive partitioning
📊 Risk scores calculated from confidence column (0-100 scale)
🔒 SQL format errors completely eliminated
🔧 Partition naming issue FIXED - now uses year=2024/month=10/ format

🔗 Next steps:
   1. Run MSCK REPAIR TABLE commands in Athena
   2. Test queries: SELECT COUNT(*) FROM docushield_analytics.risk_findings
   3. Create QuickSight datasets using these tables
   4. Build you